<a href="https://colab.research.google.com/github/eentartetekunst/NGS_bioinformatics/blob/main/HW1_Hisat2_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
! pip install -q condacolab

In [ ]:
import condacolab
condacolab.install()

In [ ]:
!conda install -y -c bioconda samtools

In [ ]:
! sudo apt install bedtools 

In [ ]:
! conda install -c bioconda regtools 

## import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! mkdir reads

In [11]:
! mv *.fq.gz content reads

mv: cannot stat 'content': No such file or directory


In [12]:
! ls reads

B14.5.fq.gz  B17.5.fq.gz  C15.5.fq.gz  C20.fq.gz
B15.5.fq.gz  B20.fq.gz	  C17.5.fq.gz  C34.fq.gz


## 2) install hisat2

In [ ]:
! sudo apt-get update

In [ ]:
! sudo apt-get -y install hisat2

## 3) download chr19 sequence and its annotation

In [ ]:
! wget http://ftp.ensembl.org/pub/release-107/fasta/mus_musculus/dna/Mus_musculus.GRCm39.dna.chromosome.19.fa.gz #chr19 sequence
! wget http://ftp.ensembl.org/pub/release-107/gtf/mus_musculus/Mus_musculus.GRCm39.107.gtf.gz # annotation
! gzip -d Mus_musculus.GRCm39.dna.chromosome.19.fa.gz
! gzip -d Mus_musculus.GRCm39.107.gtf.gz

## 4) select chr 19 from the annotation

In [16]:
! touch chr19-annotation 
! grep -P '^19\t' /content/Mus_musculus.GRCm39.107.gtf > chr19-annotation 

In [17]:
!ls -lh

total 918M
-rw-r--r-- 1 root root  27M Oct 16 11:03 chr19-annotation
-rw-r--r-- 1 root root  16K Oct 16 10:52 condacolab_install.log
-rw-r--r-- 1 root root 831M Jun  4 09:40 Mus_musculus.GRCm39.107.gtf
-rw-r--r-- 1 root root  60M Jun  4 08:49 Mus_musculus.GRCm39.dna.chromosome.19.fa
drwxr-xr-x 2 root root 4.0K Oct 16 11:01 reads
drwxr-xr-x 1 root root 4.0K Oct  7 13:35 sample_data


In [18]:
! head -n2 chr19-annotation 

19	ensembl	gene	56265950	56266077	.	-	.	gene_id "ENSMUSG00000077223"; gene_version "3"; gene_name "Gm22271"; gene_source "ensembl"; gene_biotype "snoRNA";
19	ensembl	transcript	56265950	56266077	.	-	.	gene_id "ENSMUSG00000077223"; gene_version "3"; transcript_id "ENSMUST00000104035"; transcript_version "3"; gene_name "Gm22271"; gene_source "ensembl"; gene_biotype "snoRNA"; transcript_name "Gm22271-201"; transcript_source "ensembl"; transcript_biotype "snoRNA"; tag "basic"; transcript_support_level "NA (assigned to previous version 1)";


## 5) build index for chr 19

In [ ]:
! hisat2-build Mus_musculus.GRCm39.dna.chromosome.19.fa new-index
# everything is good with this index, but no reads on exon junctions

In [ ]:
# run after running extract ss and exons 
! hisat2-build --ss chr19.ss --exon chr19.exons Mus_musculus.GRCm39.dna.chromosome.19.fa ss-index

## map all reads on chr19 sequence (no trimming, known splice sites)
use --known-splicesite-infile:
With this mode, you can provide a list of known splice sites, which HISAT makes use of to align reads with small anchors.
You can create such a list using python extract_splice_sites.py genes.gtf > splicesites.txt, where extract_splice_sites.py is included in the HISAT package, genes.gtf is a gene annotation file, and splicesites.txt is a list of splice sites with which you provide HISAT in this mode

## install extract_splice_sites.py and extract_exons.py

In [22]:
! wget https://raw.githubusercontent.com/dekoning-lab/ezmap/master/tools/HISAT2/hisat2-2.0.5-Linux/extract_splice_sites.py
! chmod a+x extract_splice_sites.py
! ./extract_splice_sites.py

--2022-10-16 11:06:05--  https://raw.githubusercontent.com/dekoning-lab/ezmap/master/tools/HISAT2/hisat2-2.0.5-Linux/extract_splice_sites.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5021 (4.9K) [text/plain]
Saving to: ‘extract_splice_sites.py’

extract_splice_site 100%[===================>]   4.90K  --.-KB/s    in 0s      

2022-10-16 11:06:05 (57.6 MB/s) - ‘extract_splice_sites.py’ saved [5021/5021]

usage: extract_splice_sites.py [-h] [-v] [gtf_file]

Extract splice junctions from a GTF file

positional arguments:
  gtf_file       input GTF file (use "-" for stdin)

optional arguments:
  -h, --help     show this help message and exit
  -v, --verbose  also print some statistics to stderr


In [23]:
! wget https://raw.githubusercontent.com/dekoning-lab/ezmap/master/tools/HISAT2/hisat2-2.0.5-Linux/extract_exons.py
! chmod a+x extract_exons.py
! ./extract_exons.py

--2022-10-16 11:06:11--  https://raw.githubusercontent.com/dekoning-lab/ezmap/master/tools/HISAT2/hisat2-2.0.5-Linux/extract_exons.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5597 (5.5K) [text/plain]
Saving to: ‘extract_exons.py’

extract_exons.py    100%[===================>]   5.47K  --.-KB/s    in 0s      

2022-10-16 11:06:11 (62.8 MB/s) - ‘extract_exons.py’ saved [5597/5597]

usage: extract_exons.py [-h] [-v] [gtf_file]

Extract exons from a GTF file

positional arguments:
  gtf_file       input GTF file (use "-" for stdin)

optional arguments:
  -h, --help     show this help message and exit
  -v, --verbose  also print some statistics to stderr


## extract splice sites and exon's locations  from the annotation

In [83]:
! ./extract_splice_sites.py chr19-annotation > chr19.ss # extract sites from the annotation
! ! ./extract_exons.py chr19-annotation > chr19.exons 

In [25]:
! head -n2 chr19.ss 

19	3103277	3109879	-
19	3109990	3119366	-


In [26]:
! head -n2 chr19.exons

19	3103070	3103277	-
19	3109879	3109990	-


## map reads on chr19 with no trimming, with known splice sites

---



## 6) Прокартируйте все fq файлы (начните с одного) на 19ую хромосому при помощи hisat2 не допуская обрезания ридов и сообщив hisat2 координаты сайтов сплайсинга 

In [ ]:
%%shell 
# reads_list='B20'
reads_list='C14.5 B34 B14.5 C15.5 C17.5 B17.5 C34 B20 B15.5 C20' 
index='ss-index' # with splicing sites and exon coordinates; new-index - without 
fq='/content/reads'

for i in ${reads_list}
do hisat2 --summary-file ${i}.log --no-softclip --no-unal -x ${index} -U ${fq}/${i}.fq.gz --known-splicesite-infile chr19.ss | samtools view -Sb - > ${i}.bam 
done

## 7) Выберите случайно один образец

## 8)Сколько ридов картируется в регион 19:12485000-12490000 в этом образце? 

In [129]:
! samtools sort -o B20.s.bam B20.bam #coordinate sorting

In [130]:
! samtools index B20.s.bam # Generate BAI-format index for BAM files

In [132]:
! samtools view B20.s.bam 19:12485000-12490000 

HISEQ:129:C2LRCACXX:1:2110:5448:97880	272	19	12479312	0	42M56847N42M50311N17M	*	0	0	TCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTTGTGGGGAG	###DDDDDDDDDDBB@BBDDDDDBB@B??<?B@B<B@B@DD@=<BBDDCCDFEFHHHJJJIGJJJIJIHDGGIJJJIJJIIJIIJIG=GHGHFFFFFFCC@	AS:i:-16	ZS:i:-16	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:101	YT:Z:UU	XS:A:+	NH:i:4
HISEQ:129:C2LRCACXX:1:2110:5448:97880	272	19	12479314	0	42M56845N42M50311N17M	*	0	0	TCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTTGTGGGGAG	###DDDDDDDDDDBB@BBDDDDDBB@B??<?B@B<B@B@DD@=<BBDDCCDFEFHHHJJJIGJJJIJIHDGGIJJJIJJIIJIIJIG=GHGHFFFFFFCC@	AS:i:-16	ZS:i:-16	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:101	YT:Z:UU	XS:A:+	NH:i:4
HISEQ:129:C2LRCACXX:1:2110:5448:97880	272	19	12479316	0	42M56843N42M50311N17M	*	0	0	TCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCTCGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTGTTGTGGGGAG	###DDDDDDDDDDBB@BBDDDDDBB@B??<?B@B<B@B@DD@=<BBDDCCDFEFHHHJJJIGJJJIJIHD

Риды с кодом 272 не уникальные (non-primary), то есть всего уникальных рида 4 

## 9) Сколько из них картируются только в одно место генома?


In [133]:
! samtools view B20.s.bam 19:12485000-12490000 -d NH:1 -c  
#  риды, которые картировлаись на одно место в данном участке 
# -c Instead of printing the alignments, only print the total number. 

3


In [134]:
! samtools view B20.s.bam 19:12485000-12490000  -d NH:1 

HISEQ:129:C2LRCACXX:1:2108:20530:65021	16	19	12486735	60	101M	*	0	0	TTTAAAATCACCCAATAGACTGACTTGCCAGCGTGCCTGTAAGGAGTTATCTAGAATCAGTTAACTAAGGTAGGAATACCCACCCTAACCTTGGGCAGCAC	:CCDEEDB@@DCDEECDCCDEDCBC@DDFFHHHIIIIIIIIIIIIIIIIIFIIIIIIIIIIIIIIIIIIIIIIIHE<?IGF?IIIIGGHHHHHFFFFFCCC	AS:i:0	XN:i:0	XM:i:0	XO:i:0	XG:i:0	NM:i:0	MD:Z:101	YT:Z:UU	NH:i:1
HISEQ:129:C2LRCACXX:1:1204:10203:87648	16	19	12487196	60	101M	*	0	0	TGTATTGACATCTTAAACTATTTCAAATATACCCCAACCTCAGCTAATCTGGGTTACAAACCCCCTAAAATTCAAATAACATCTCATTTGAATGTCCAGAC	@C:EEEDEDDDCDEEFCDDDDDDEFFFDDBBDDFFECHHHIJJJJJJJJJJJJIIJJIHFIJJJJIJJJJIIJJJJIIJJIHHIJJJJHHHHHFFFFFCCC	AS:i:-5	XN:i:0	XM:i:1	XO:i:0	XG:i:0	NM:i:1	MD:Z:5C95	YT:Z:UU	NH:i:1
HISEQ:129:C2LRCACXX:1:2104:14666:2583	16	19	12487196	60	101M	*	0	0	TGTATTGACATCTTAAACTATTTCAAATATACCCCAACCTCAGCTAATCTGGGTTACAAACCCCCTAAAATTCAAATAACATCTCATTTGAATGTCCAGAC	CDEEEEDEDDDDDEEDDDDDDDEEFFFEDBDDDFFEHHHHJJJJJJJIJJJJJJJJIHHFJJJJIJJJJIIHIJJJJJHIIHJIJJJJHHHHHFFFFFCCC	AS:i:-5	XN:i:0	XM:i:1	XO:i:0	XG:i:0	NM:i:1	MD:Z:5C9

три рида

## 10) Сколько ридов картровалось без замен? Сколько с 1, 2 и т. д. заменами?

In [135]:
! samtools view B20.s.bam 19:12485000-12490000 -d NM:1 -c # с 1 заменой 

2


In [136]:
! samtools view B20.s.bam 19:12485000-12490000  -d NM:2 -c # с 2 заменами

0


Риды, картировавшиеся на всю хромосому с i заменами:

In [119]:
%%shell
list_my_i='1 2 3 4 5 6 7 8 9 10 11 12'
for i in ${list_my_i}
do samtools view B20.s.bam  -d NM:${i} -c # с i заменами -- картирование на всю хромосому
done

105303
26239
11657
4075
2136
1253
561
251
58
28
0
0


## 11) Сколько ридов картировалось на экзон-экзонные границы? 

https://regtools.readthedocs.io/en/latest/commands/junctions-extract/ 

The junctions extract command can be used to extract exon-exon junctions from an RNAseq BAM file. The output is a BED file in the BED12 format

In [137]:
! samtools index B20.s.bam 

In [138]:
# /content/B14.5.bam.bai path to coordinate-sorted file
! regtools junctions extract -o B20.bed -s XS B20.s.bam 


Program:	regtools
Version:	0.5.2
Minimum junction anchor length: 8
Minimum intron length: 70
Maximum intron length: 500000
Alignment: B20.s.bam
Output file: B20.bed



In [139]:
! head B20.bed

19	3133283	3137373	JUNC00000001	1	+	3133283	3137373	255,0,0	2	58,43	0,4047
19	3277734	3277911	JUNC00000002	1	+	3277734	3277911	255,0,0	2	84,17	0,160
19	3311559	3312030	JUNC00000003	2	-	3311559	3312030	255,0,0	2	76,56	0,415
19	3312071	3314887	JUNC00000004	2	-	3312071	3314887	255,0,0	2	73,77	0,2739
19	3315609	3316434	JUNC00000005	4	-	3315609	3316434	255,0,0	2	56,99	0,726
19	3316371	3317302	JUNC00000006	3	-	3316371	3317302	255,0,0	2	88,61	0,870
19	3317262	3318324	JUNC00000007	1	-	3317262	3318324	255,0,0	2	74,27	0,1035
19	3318371	3318756	JUNC00000008	5	-	3318371	3318756	255,0,0	2	45,97	0,288
19	3318772	3321588	JUNC00000009	2	-	3318772	3321588	255,0,0	2	70,63	0,2753
19	3321609	3322913	JUNC00000010	2	-	3321609	3322913	255,0,0	2	91,26	0,1278


In [ ]:
! regtools junctions annotate -o B20.exonexon.reads B20.bed Mus_musculus.GRCm39.dna.chromosome.19.fa chr19-annotation 

In [141]:
! head B20.exonexon.reads 
# score - The number of reads supporting the junction. [integer]

chrom	start	end	name	score	strand	splice_site	acceptors_skipped	exons_skipped	donors_skipped	anchor	known_donor	known_acceptor	known_junction	gene_names	gene_ids	transcripts
19	3133341	3137331	JUNC00000001	1	+	GT-AG	0	0	0	N	0	0	0	NA	NA	NA
19	3277818	3277895	JUNC00000002	1	+	GT-AG	0	0	0	N	0	0	0	NA	NA	NA
19	3311635	3311975	JUNC00000003	2	-	GT-AG	0	0	0	DA	1	1	1	Ighmbp2	ENSMUSG00000024831	ENSMUST00000025751,ENSMUST00000119292
19	3312144	3314811	JUNC00000004	2	-	GT-AG	0	0	0	DA	1	1	1	Ighmbp2	ENSMUSG00000024831	ENSMUST00000025751,ENSMUST00000119292
19	3315665	3316336	JUNC00000005	4	-	GT-AG	0	0	0	DA	1	1	1	Ighmbp2	ENSMUSG00000024831	ENSMUST00000025751,ENSMUST00000119292,ENSMUST00000143079,ENSMUST00000154537
19	3316459	3317242	JUNC00000006	3	-	GT-AG	0	0	0	DA	1	1	1	Ighmbp2	ENSMUSG00000024831	ENSMUST00000025751,ENSMUST00000119292,ENSMUST00000143079,ENSMUST00000154537
19	3317336	3318298	JUNC00000007	1	-	GT-AG	1	0	0	DA	1	1	1	Ighmbp2	ENSMUSG00000024831	ENSMUST00000025751,ENSMUST00000119292,ENSMUST000

In [142]:
! cat B20.exonexon.reads | awk '($2 <= 12490000 && $3 >= 12485000)' 
# риды из данного интервала, в которых есть экзон-экзонная граница
# таких ридов нет, если индекс строить без координат сплайсинга и экзонов однако, если  подавать сайтты сплайсинга при построении индекса, их 4:

19	12479353	12536201	JUNC00002608	1	+	TC-GT	1	0	2	N	0	0	0	NA	NA	NA
19	12479355	12536201	JUNC00002610	1	+	TC-GT	1	0	2	N	0	0	0	NA	NA	NA
19	12479357	12536201	JUNC00002611	1	+	TC-GT	1	0	2	N	0	0	0	NA	NA	NA
19	12479359	12536201	JUNC00002612	1	+	TG-GT	1	0	2	N	0	0	0	NA	NA	NA


## 12) Перечислите координаты всех интронов в данном интервале подтверждённых хотя бы одним ридом в формате: chr:from-to coverage

In [143]:
! samtools view B20.s.bam -b 19:12485000-12490000 > targetB20.s.bam # choose target region

In [144]:
! bedtools bamtobed -i targetB20.s.bam > targetB20.s.bed 

In [145]:
! head targetB20.s.bed # reads in target region 

19	12479311	12586570	HISEQ:129:C2LRCACXX:1:2110:5448:97880	0	-
19	12479313	12586570	HISEQ:129:C2LRCACXX:1:2110:5448:97880	0	-
19	12479315	12586570	HISEQ:129:C2LRCACXX:1:2110:5448:97880	0	-
19	12479317	12586570	HISEQ:129:C2LRCACXX:1:2110:5448:97880	0	-
19	12486734	12486835	HISEQ:129:C2LRCACXX:1:2108:20530:65021	60	-
19	12487195	12487296	HISEQ:129:C2LRCACXX:1:1204:10203:87648	60	-
19	12487195	12487296	HISEQ:129:C2LRCACXX:1:2104:14666:2583	60	-


### bedtools complement -i exons.bed -g genome.txt > non-exonic.bed -- to get intron coordinates 

In [146]:
! head /content/chr19.exons

19	3103070	3103277	-
19	3109879	3109990	-
19	3115710	3118148	-
19	3119366	3119439	-
19	3122687	3122952	-
19	3125713	3125884	-
19	3184644	3184958	+
19	3186983	3187089	-
19	3192759	3193064	+
19	3195867	3196264	-


Если вычести из всех ридов те, что попадают на экзоны, мы получим покрытие интронов:

In [ ]:
! bedtools subtract -a targetB20.s.bed -b chr19.exons 

Координаты интронных ридов:

In [148]:
! bedtools subtract -a targetB20.s.bed -b chr19.exons  > introns.bed 
! awk -F"\t" '$5 > 0 { print $2"\t"$3 }' introns.bed 
# retrieve introns in the region, covered by at least one read 
# $5 > 0 stands for coverage score

12486734	12486835
12487195	12487296
12487195	12487296
